In [525]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
# deep learning
import torch
import pandas as pd
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader 
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchmetrics import Accuracy

In [526]:
monk1_train = pd.read_csv('data/monks-1.train',header=None,sep=" ")
monk1_test = pd.read_csv('data/monks-1.test',header=None,sep=" ")

In [527]:
monk1_train.drop(0,axis=1,inplace=True)
monk1_train.drop(8,axis=1,inplace=True)

monk1_test.drop(0,axis=1,inplace=True)
monk1_test.drop(8,axis=1,inplace=True)

In [528]:
xtr = monk1_train.iloc[:,1:]
ytr = monk1_train.iloc[:,0]

xts = monk1_test.iloc[:,1:]
yts = monk1_test.iloc[:,0]

In [529]:
type(xtr)

pandas.core.frame.DataFrame

In [530]:
scaler = StandardScaler()
xtr = scaler.fit_transform(xtr)
xts = scaler.fit_transform(xts)

In [531]:
type(xtr)

numpy.ndarray

In [532]:
xtr, xvl, ytr, yvl = train_test_split(xtr, ytr, test_size=0.10, random_state=42)

In [533]:
ytr = ytr.to_numpy()
yvl = yvl.to_numpy()

In [534]:
type(yvl)

numpy.ndarray

In [535]:
class ClassificationMonk(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden1, learning_rate):
        super(ClassificationMonk, self).__init__()
        # First hidden layer
        self.linear1 = nn.Linear(input_dim, hidden1)
        self.act1 = nn.ReLU()
        # Second hidden layer
        self.linear2 = nn.Linear(hidden1, hidden1)
        self.act2 = nn.ReLU()
        # Third hidden layer
        self.linear3 = nn.Linear(hidden1,output_dim)
        self.act3 = nn.Sigmoid()
        self.loss_fun = nn.MSELoss()
        self.learning_rate = learning_rate
    
    def forward(self, X):
        #Input to the first hidden layer
        X = self.linear1(X)
        X = self.act1(X)
        # Second hidden layer
        X = self.linear2(X)
        X = self.act2(X)
        # Third hidden layer
        X = self.linear3(X)
        X = self.act3(X)
        return X

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        X, y = train_batch  #1
        X = X.type(torch.float32)
        y = y.type(torch.float32)  #2
        # forward pass
        y_pred = self.forward(X).squeeze()  #3 
        # compute loss
        loss = self.loss_fun(y_pred, y)  
        self.log_dict({'train_loss': loss}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, validation_batch, batch_idx):
        X, y = validation_batch
        #y = y.type(torch.float32)
        X = X.type(torch.float32)
        # forward pass
        y_pred = self.forward(X).squeeze()        
        # compute metrics       
        print(y_pred) 
        accuracy = Accuracy()
        acc = accuracy(y_pred, y)
        loss = self.loss_fun(y_pred[0], y[0])
        self.log_dict({'validation_loss': loss, 'accuracy': acc}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        #y = y.type(torch.float32)
        X = X.type(torch.float32)
        # forward pass
        y_pred = self.forward(X).squeeze()        
        # compute metrics       
        print(y_pred) 
        accuracy = Accuracy()
        acc = accuracy(y_pred, y)
        loss = self.loss_fun(y_pred[0], y[0])
        self.log_dict({'test_loss': loss, 'accuracy': acc}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

In [536]:
hidden1 = 20  
max_epochs = 500  
lr = 0.01
train_batch_size = 20  
validation_batch_size = 20  
test_batch_size = 20 

In [537]:
# model instance
input_dim = xtr.shape[1]
model = ClassificationMonk(input_dim=input_dim, output_dim=1, hidden1=hidden1, learning_rate=lr)

In [538]:
class ClassificationMonkDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [539]:
train_loader = DataLoader(dataset = ClassificationMonkDataset(xtr, ytr), batch_size=train_batch_size)
validation_loader = DataLoader(dataset = ClassificationMonkDataset(xvl, yvl), batch_size=validation_batch_size)
test_loader = DataLoader(dataset = ClassificationMonkDataset(xts, yts), batch_size=test_batch_size)

In [540]:
# early stopping
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=0.0001, patience=10, verbose=True, mode="min")

In [541]:
trainer = pl.Trainer(accelerator='cpu', devices=1, max_epochs=max_epochs, callbacks=[early_stop_callback], log_every_n_steps=5,auto_lr_find=True)  
trainer.tune(model=model, train_dataloaders=train_loader) #for auto_lr_finder

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/dylan/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/Users/dylan/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.04365158322401657
Restoring states from the checkpoint path at /Users/dylan/Desktop/ML-Project/.lr_find_59bc2518-b4d5-4187-b72e-949fe041d609.ckpt
Restored all states from the checkpoint file at /Users/dylan/Desktop/ML-Project/.lr_find_59bc2518-b4d5-4187-b72e-949fe041d609.ckpt


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x29c13e850>}

In [542]:
trainer.fit(model=model, train_dataloaders=train_loader) 


  | Name     | Type    | Params
-------------------------------------
0 | linear1  | Linear  | 140   
1 | act1     | ReLU    | 0     
2 | linear2  | Linear  | 420   
3 | act2     | ReLU    | 0     
4 | linear3  | Linear  | 21    
5 | act3     | Sigmoid | 0     
6 | loss_fun | MSELoss | 0     
-------------------------------------
581       Trainable params
0         Non-trainable params
581       Total params
0.002     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Metric train_loss improved. New best score: 0.243
Metric train_loss improved by 0.060 >= min_delta = 0.0001. New best score: 0.183
Metric train_loss improved by 0.034 >= min_delta = 0.0001. New best score: 0.149
Metric train_loss improved by 0.038 >= min_delta = 0.0001. New best score: 0.111
Metric train_loss improved by 0.027 >= min_delta = 0.0001. New best score: 0.084
Metric train_loss improved by 0.025 >= min_delta = 0.0001. New best score: 0.059
Metric train_loss improved by 0.020 >= min_delta = 0.0001. New best score: 0.039
Metric train_loss improved by 0.019 >= min_delta = 0.0001. New best score: 0.019
Metric train_loss improved by 0.012 >= min_delta = 0.0001. New best score: 0.007
Metric train_loss improved by 0.003 >= min_delta = 0.0001. New best score: 0.004
Metric train_loss improved by 0.001 >= min_delta = 0.0001. New best score: 0.002
Monitored metric train_loss did not improve in the last 10 records. Best score: 0.002. Signaling Trainer to stop.


In [546]:
trainer.test(model=model, dataloaders=validation_loader)

/Users/dylan/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

tensor([1.0371e-03, 1.2767e-07, 1.0000e+00, 1.0000e+00, 1.3615e-03, 1.1458e-06,
        2.2357e-05, 9.7660e-01, 1.0000e+00, 9.9996e-01, 3.9034e-10, 1.0000e+00,
        3.0495e-07])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy                    1.0
        test_loss         1.0755005632745451e-06
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.0755005632745451e-06, 'accuracy': 1.0}]

In [547]:
# no good estimation of model, just testing
trainer.test(model=model, dataloaders=test_loader)

Testing: 0it [00:00, ?it/s]

tensor([1.0000, 1.0000, 0.9934, 0.9983, 0.9065, 0.9973, 0.8012, 0.9904, 1.0000,
        1.0000, 0.9568, 0.9887, 0.2475, 0.9771, 0.1141, 0.9692, 1.0000, 1.0000,
        0.9083, 0.9961])
tensor([0.4295, 0.9816, 0.2289, 0.9829, 1.0000, 1.0000, 1.0000, 0.9999, 0.9993,
        0.9999, 0.9987, 0.9999, 1.0000, 1.0000, 0.9999, 0.9984, 0.9972, 0.9986,
        0.9733, 0.9976])
tensor([1.0000e+00, 1.0000e+00, 9.9847e-01, 9.9816e-01, 9.6983e-01, 9.9470e-01,
        8.2882e-01, 9.8847e-01, 1.0000e+00, 1.0000e+00, 1.1950e-05, 2.5811e-04,
        7.7199e-06, 1.9543e-04, 1.0448e-05, 2.1890e-04, 1.0000e+00, 1.0000e+00,
        8.3878e-06, 2.7646e-04])
tensor([1.2547e-06, 1.1987e-04, 8.4945e-07, 1.3237e-04, 1.0000e+00, 1.0000e+00,
        1.7411e-05, 1.3430e-03, 1.3302e-06, 1.8705e-04, 1.3442e-06, 1.8297e-04,
        1.0000e+00, 1.0000e+00, 6.6016e-03, 1.7659e-02, 6.8745e-04, 2.8695e-02,
        5.1183e-03, 2.3396e-01])
tensor([1.0000e+00, 1.0000e+00, 3.3505e-03, 3.9531e-03, 6.1244e-05, 1.6376e-03,
    

[{'test_loss': 0.015096260234713554, 'accuracy': 0.9513888955116272}]